# Question Answering Challenge

### Using a Wikipedia dump for offline retrieval

As specified in https://en.wikipedia.org/wiki/Wikipedia:Database_download, we download the Polish Wikipedia dump from https://dumps.wikimedia.org/.

In [ ]:
import requests
from tqdm import tqdm

url = 'https://dumps.wikimedia.org/plwiki/latest/plwiki-latest-pages-articles.xml.bz2'

response = requests.get(url, stream=True)
if response.status_code == 200:
    file_name = 'plwiki-latest-pages-articles.xml.bz2'
    total_size = int(response.headers.get('content-length', 0))
    chunk_size = 1024  # 1 KB
    with open(file_name, 'wb') as file, tqdm(
        desc=file_name,
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                file.write(chunk)
                bar.update(len(chunk))
    print(f'Wikipedia dump downloaded successfully: {file_name}')
else:
    print(f"Failed to download the Wikipedia dump. HTTP Status Code: {response.status_code}")


### Extracting data from the Wikipedia dump using WikiExtractor

Run the following command to extract articles from the dump into an `extracted/` folder:
```bash
wikiextractor --json plwiki-latest-pages-articles.xml.bz2 -o extracted
```

### Processing the extracted data and indexing it into Elasticsearch

In [1]:
from elasticsearch import Elasticsearch

INDEX_NAME = "wiki_index"

es = Elasticsearch("http://localhost:9200")
es.indices.delete(index=INDEX_NAME, ignore=[400, 404])

C:\Users\szkar\AppData\Local\Temp\ipykernel_25284\2279654839.py:6: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=INDEX_NAME, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [ ]:
import os
import json
import logging
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk

INDEX_NAME = "wiki_index"

INDEX_BODY = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "paragraph_number": {"type": "integer"},
            "content": {"type": "text"}
        }
    }
}

es = Elasticsearch("http://localhost:9200")

# UTWORZENIE INDEKSU (jeśli nie istnieje)
if not es.indices.exists(index=INDEX_NAME):
    es.indices.create(index=INDEX_NAME, body=INDEX_BODY)
    # Wyłączamy odświeżanie i replikę na czas indeksowania (przyspieszy to import)
    es.indices.put_settings(
        index=INDEX_NAME,
        body={
            "index": {
                "refresh_interval": "-1",
                "number_of_replicas": 0
            }
        }
    )
    print(f"Created index '{INDEX_NAME}' with temporary settings (no refresh, 0 replicas).")

# DEFINICJA GENERATORA DOKUMENTÓW
def generate_actions(dump_dir, index_name):
    """
    Generator dla parallel_bulk. Dla każdego pliku wiki_... wyciąga artykuły,
    dzieli je na paragrafy i yielduje dokumenty do zindeksowania.
    """
    for root, _, files in os.walk(dump_dir):
        for file in files:
            if file.startswith('wiki_'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        article = json.loads(line)
                        title = article.get('title', '')
                        content = article.get('text', '')

                        paragraphs = content.split("\n")
                        for i, paragraph in enumerate(paragraphs):
                            paragraph = paragraph.strip()
                            if paragraph:
                                yield {
                                    "_index": index_name,
                                    "_source": {
                                        "title": title,
                                        "paragraph_number": i,
                                        "content": paragraph
                                    }
                                }

# WYWOŁANIE parallel_bulk DO MASOWEGO INDEKSOWANIA
actions = generate_actions("extracted", INDEX_NAME)

successes = 0
for ok, resp in parallel_bulk(
    client=es,
    actions=actions,
    thread_count=4,      # liczba wątków
    chunk_size=500,      # wielkość jednej paczki dokumentów
    max_chunk_bytes=10 * 1024 * 1024  # ~10 MB na paczkę
):
    if not ok:
        logging.error(f"Error indexing chunk: {resp}")
    else:
        successes += 1

print(f"Successfully processed {successes} chunks of data.")

# PRZYWRACANIE NORMALNYCH USTAWIEŃ
es.indices.put_settings(
    index=INDEX_NAME,
    body={
        "index": {
            "refresh_interval": "1s",
            "number_of_replicas": 1
        }
    }
)
print(f"Indexing complete. Restored normal settings for '{INDEX_NAME}'.")


### Retrieve keywords from question

In [2]:
import spacy

nlp = spacy.load('pl_core_news_sm')
def get_keywords(question):
    doc = nlp(question)
    keywords = [token.text for token in doc if not token.is_stop] # and token.pos_ in ["NOUN", "VERB", "NUM", "PROPN"]
    return " ".join(keywords)

### Searching and answer extraction

In [10]:
from transformers import pipeline
from elasticsearch import Elasticsearch

# Inicjalizacja Elasticsearch i pipeline QA
es = Elasticsearch("http://localhost:9200")
#qa_pipeline = pipeline("question-answering", model="radlab/polish-qa-v2")
qa_pipeline = pipeline("question-answering", model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2", device=0)
# qa_pipeline = pipeline("question-answering", model="sdadas/polish-roberta-large-v2") <- super bad
# qa_pipeline = pipeline("question-answering", model="sdadas/polish-gpt2-xl") <- super bad as well

Some weights of the model checkpoint at henryk/bert-base-multilingual-cased-finetuned-polish-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


## Ollama requests

In [16]:
import requests
import json

def query_ollama(prompt, model="llama3.1", server_url="http://localhost:11434"):
    url = f"{server_url}/api/generate"
    payload = {
        "model": model,
        "prompt": prompt
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers, stream=True)
    
    if response.status_code == 200:
        # Przetwarzanie strumienia odpowiedzi
        result = ""
        for chunk in response.iter_lines():
            if chunk:
                data = json.loads(chunk.decode('utf-8'))
                if "response" in data:
                    result += data["response"]
                if data.get("done"):
                    break
        return result
    else:
        raise Exception(f"Błąd: {response.status_code}, {response.text}")


In [17]:
# Przykład użycia:
prompt = "Opisz funkcjonalność modelu LLama 3.1 w jednym zdaniu."
try:
    answer = query_ollama(prompt)
    print("Model odpowiedział:", answer)
except Exception as e:
    print("Wystąpił błąd:", e)

Model odpowiedział: LLaMA 3 jest wielowarstwowym modelem języka, który może wykonać takie operacje jak generowanie tekstu, interpretacja języka naturalnego i odpowiedzi na pytania.


## Ultimate parse

In [403]:
import spacy
import re

nlp = spacy.load('pl_core_news_sm')
def extract_symbols(text):
    pattern = r'\b[A-Z][a-zA-Z]?\b'
    symbols = re.findall(pattern, text)
    return symbols

def is_important_token(token, text, symbols_set):
    return (token.pos_ in ["NOUN", "VERB", "NUM", "ADJ", "ADV", "DET"]
            and token.text not in symbols_set
            and not token.is_stop)

def get_keywords(question):
    doc = nlp(question)
    all_symbols = extract_symbols(question)
    symbols_set = {sym for sym in all_symbols if not nlp(sym)[0].is_stop}
    
    keywords = [
        token.lemma_ for token in doc
        if is_important_token(token, question, symbols_set)
    ]
    return list(symbols_set), list(set(keywords))

def extractentities(question):
    doc = nlp(question)
    entities = [(ent.text, ent.lemma_, ent.label_) for ent in doc.ents]
    return entities

def extract_and_remove_quotes(text):
    pattern = r"'(.*?)'"
    quotes = re.findall(pattern, text)
    modified_text = re.sub(pattern, '', text)
    return quotes, modified_text

def ultimate_parse(question):
    entities = extractentities(question)
    text = question
    for ent_text in [t for (t, _, _) in entities]:
        text = text.replace(ent_text, "")
    quotes, modified_text = extract_and_remove_quotes(text)
    lema_entities = [l for (_, l, _) in entities]
    symbols, tokens = get_keywords(modified_text)

    return quotes, lema_entities, symbols, tokens

In [404]:
print(ultimate_parse("W jakim państwie znajduje się Bombaj"))
print(ultimate_parse("Jak nazywa się pierwiastek o symbolu Bq"))
print(ultimate_parse("Ile pełnych tygodni ma rok kalendarzowy"))
print(ultimate_parse("W którym państwie rozpoczyna się akcja powieści 'W pustyni i w puszczy'?"))
print(ultimate_parse("Jak nazywał się sztywny kapelusz męski o zaokrąglonej główce i wąskim lekko uniesionym rondelku, modny w drugiej połowie XIX wieku"))
print(ultimate_parse("Jak nazywa się wypukła albo wklęsła powierzchnia cieczy w pobliżu ścianek naczynia"))
print(ultimate_parse("Jak brzmi ulubione powiedzonko klucznika Horeszków Gerwazego?"))
question = "Którego gazu jest najwięcej w powietrzu?"#"Kto stworzył Tomb Raider?"
print(ultimate_parse(question))
# print(get_keywords(question))
# answer_question(question, es, INDEX_NAME)

([], ['Bombaj'], [], ['znajdować', 'państwo', 'jaki'])
([], [], ['Bq'], ['symbol', 'nazywać', 'pierwiastek'])
([], [], [], ['pełny', 'rok', 'kalendarzowy', 'tydzień'])
(['W pustyni i w puszczy'], [], [], ['państwo', 'akcja', 'powieść', 'rozpoczynać'])
([], ['XIX wiek'], [], ['sztywny', 'modny', 'rondelke', 'wąski', 'zaokrąglony', 'unieść', 'kapelusz', 'drugi', 'Męski', 'nazywać', 'lekko', 'główka', 'połowa'])
([], [], [], ['ciecz', 'wklęsła', 'pobliże', 'ścianek', 'wypuknąć', 'nazywać', 'naczynie', 'powierzchnia'])
([], ['Horeszk Gerwaz'], [], ['ulubiony', 'brzmieć', 'klucznika', 'powiedzonko'])
([], [], [], ['powietrze', 'najwięcej', 'gaz'])


In [423]:
INDEX_NAME = "wiki_index"

# 2. Funkcja pobierająca akapity z ES
def retrieve_paragraphs(question, es_client, index_name, size=10):
    quotes, lema_entities, symbols, tokens = ultimate_parse(question)

    must_clauses = []
    for quote in quotes:
        must_clauses.append({"match_phrase": {"content": quote}})

    should_clauses = []
    if lema_entities:
        should_clauses.append({
            "multi_match": {
                "query": " ".join(lema_entities),
                "fields": ["title", "content"],
                "fuzziness": "AUTO",
                "boost": 2.0
            }
        })
    if symbols:
        should_clauses.append({
            "multi_match": {
                "query": " ".join(symbols),
                "fields": ["title", "content"],
                "fuzziness": "AUTO",
                "boost": 1.5
            }
        })
    if tokens:
        should_clauses.append({
            "multi_match": {
                "query": " ".join(tokens),
                "fields": ["title", "content"],
                "fuzziness": "AUTO",
                "boost": 1.0
            }
        })

    query = {
        "query": {
            "bool": {
                "must": must_clauses,
                "should": should_clauses,
                "minimum_should_match": "40%"
            }
        },
        "size": size
    }

    response = es_client.search(index=index_name, body=query)

    paragraphs = []
    for hit in response["hits"]["hits"]:
        paragraphs.append(hit["_source"]["content"])
    return paragraphs



In [424]:
def get_best_answer_ollama(question, paragraphs):
    # Zapytanie do modelu LLama
    prompt = ""

    for i, paragraph in enumerate(paragraphs):
        prompt += f"\n\n{i+1}: {paragraph}"
    try:        
        prompt += "Powyżej znajduje się kilka możliwych kontekstów które mogą pomóc udzielić odpowiedzi na pytanie. "
        prompt += "Każdy w osobnym akapicie. Na ich podstawie proszę odpowiedzieć na pytanie. "
        prompt += "Uwaga, nie wszystkie konteksty pasują do zadanego pytania. "
        prompt += "Jeśli żaden kontekst nie pasuje, sam podaj odpowiedź. "
        prompt += "Jeśli to możliwe - odpowiedz jednym słowem, jeśli nie, bardzo krótko, w mniej niż 5 słowach. "
        prompt += "Nie wyjasniaj odpowiedzi ani sam nie dodawaj żadnych dodatkowych informacji. "
        prompt += "Nie pisz również którego akapitu użyłeś. Chcę tylko odpowiedź na pytanie."
        prompt += f"\n\nPytanie: {question}"
        if question.lower().startswith("czy"):
            question += " Odpowiedz koniecznie jednym słowem: tak/nie."
        answer = query_ollama(prompt)
        return answer
    except Exception as e:
        print("Wystąpił błąd:", e)
        return None

In [425]:
# test:
question = "Kto napisał Trylogię?"
paragraphs = ["Trylogia to cykl powieści fantasy autorstwa J.R.R. Tolkiena.", "Trylogia to cykl powieści autorstwa Jana Kowalskiego."]

answer = get_best_answer_ollama(question, paragraphs)
print("Model odpowiedział:", answer)

Model odpowiedział: J.R.R. Tolkien


In [426]:

# 3. Funkcja, która spośród pobranych paragrafów wybiera najlepszą odpowiedź
def get_best_answer(question, paragraphs):
    """
    Wywołuje pipeline QA dla każdego paragrafu i zwraca najlepszą odpowiedź wraz z wynikiem.
    """
    def update_best(result, current_best):
        """Aktualizuje najlepszy wynik i odpowiedź."""
        if result["score"] > current_best["score"]:
            return {"answer": result["answer"], "score": result["score"], "context": result["context"]}
        return current_best

    # Inicjalizacja najlepszych wyników dla pytania i jego słów kluczowych
    best_result = {"answer": None, "score": float("-inf"), "context": None}
    #best_result_kw = {"answer": None, "score": float("-inf"), "context": None}

    for paragraph in filter(str.strip, paragraphs):
        result = qa_pipeline(question=question, context=paragraph)
        result["context"] = paragraph
        best_result = update_best(result, best_result)

        #kw_result = qa_pipeline(question=get_keywords(question), context=paragraph)
        #kw_result["context"] = paragraph
        #best_result_kw = update_best(kw_result, best_result_kw)

    # if best_result["context"] and best_result_kw["context"]:
    #     combined_context = best_result["context"] + best_result_kw["context"]
    #     combined_result = qa_pipeline(question=question, context=combined_context)
    #     combined_result["context"] = combined_context
    #     best_result = update_best(combined_result, best_result)

    # elif best_result_kw["context"]:
    #     best_result = best_result_kw

    return best_result["answer"], best_result["score"]


In [427]:
# 4. Przykładowe wywołanie

def answer_question(question, es, index_name):
    paragraphs = retrieve_paragraphs(question, es, index_name, size=8)
    print(f"Znaleziono {len(paragraphs)} pasujących paragrafów.")
    best_answer, best_score = get_best_answer(question, paragraphs)
    ollama_answer = get_best_answer_ollama(question, paragraphs)
    
    print(f"Pytanie: {question}")
    print(f"Najlepsza odpowiedź: {best_answer}")
    print(f"Score: {best_score}")
    print(f"Odpowiedź z LLamy: {ollama_answer}")

In [428]:
question = "Którego gazu jest najwięcej w powietrzu?"#"Kto stworzył Tomb Raider?"
# print(get_keywords(question))
answer_question(question, es, INDEX_NAME)


Znaleziono 8 pasujących paragrafów.
Pytanie: Którego gazu jest najwięcej w powietrzu?
Najlepsza odpowiedź: Wentylator
Score: 0.6189001202583313
Odpowiedź z LLamy: Powietrze.


In [429]:
question = "Jak nazywa się bohaterka gier komputerowych z serii Tomb Raider?"
answer_question(question, es, INDEX_NAME)

question = "Czy w państwach starożytnych powoływani byli posłowie i poselstwa?"
paragraphs = retrieve_paragraphs(question, es, INDEX_NAME, size=10)
question_tuned = "Czy w starożytności powoływani byli posłowie i poselstwa? Tak czy nie?"
best_answer, best_score = get_best_answer(question_tuned, paragraphs)
ollama_answer = get_best_answer_ollama(question_tuned, paragraphs)

print(f"Pytanie: {question}")
print(f"Najlepsza odpowiedź: {best_answer}")
print(f"Score: {best_score}")
print(f"Odpowiedź z LLamy: {ollama_answer}")

Znaleziono 8 pasujących paragrafów.
Pytanie: Jak nazywa się bohaterka gier komputerowych z serii Tomb Raider?
Najlepsza odpowiedź: Lary Croft
Score: 0.8891009092330933
Odpowiedź z LLamy: Lara Croft.
Pytanie: Czy w państwach starożytnych powoływani byli posłowie i poselstwa?
Najlepsza odpowiedź: 1945 na szczeblu poselstw
Score: 0.04706767573952675
Odpowiedź z LLamy: Tak.


In [430]:
question = "Ile pełnych tygodni ma rok kalendarzowy?"
answer_question(question, es, INDEX_NAME)
print('-------------------')

question = "Pierwiastek symbolu Bq?"
answer_question(question, es, INDEX_NAME)

Znaleziono 8 pasujących paragrafów.
Pytanie: Ile pełnych tygodni ma rok kalendarzowy?
Najlepsza odpowiedź: 354, 384 lub 385
Score: 0.21303628385066986
Odpowiedź z LLamy: 52
-------------------
Znaleziono 8 pasujących paragrafów.
Pytanie: Pierwiastek symbolu Bq?
Najlepsza odpowiedź: bekerel
Score: 0.5004624128341675
Odpowiedź z LLamy: Bekerel.


### Evaluation

In [433]:
from difflib import SequenceMatcher
import re

def levenshtein_distance(s1, s2):
    return SequenceMatcher(None, s1, s2).ratio()

def is_textual_match(pred, gold, threshold=0.5):
    return levenshtein_distance(pred.lower(), gold.lower()) >= threshold

def is_numerical_match(pred, gold):
    pred_num = re.search(r"\d+", pred)
    gold_num = re.search(r"\d+", gold)
    if pred_num and gold_num:
        return pred_num.group() == gold_num.group()
    return False

def evaluate(in_file, expected_file):
    with open(in_file, 'r', encoding='utf-8') as file_in, open(expected_file, 'r', encoding='utf-8') as file_expected:
        questions = [line.strip() for line in file_in]
        gold_answers = [line.strip() for line in file_expected]
    
    total = 200
    print(f"Total questions: {total}")
    correct = 0

    for index, (question, gold) in enumerate(zip(questions, gold_answers)):
        if index >= total:
            # My machine cannot handle more
            break
        paragraphs = retrieve_paragraphs(question, es, INDEX_NAME, size=8)
        # best_answer, best_score = get_best_answer(question, paragraphs)
        # print(f"keywords: {get_keywords(question)}")
        # print(f"Q: {question}\nA: {best_answer}\nExpected: {gold}\n\n")
        # print(f"Score: {best_score}\n\n")
        # if best_answer is None:
        #     continue

        best_answer = get_best_answer_ollama(question, paragraphs)
        print(f"Q: {question}\nA: {best_answer}\nExpected: {gold}")

        if is_numerical_match(best_answer, gold) or is_textual_match(best_answer, gold):
            print("Correct!\n")
            correct += 1
        else:
            print("Incorrect!\n")

    print(f"Correct: {correct}")
    accuracy = float(correct) / float(total)
    print(f"Accuracy: {accuracy:.2%}")

DEV_0_IN = "data/dev-0/in.tsv"
DEV_0_EXPECTED = "data/dev-0/expected.tsv"
evaluate(DEV_0_IN, DEV_0_EXPECTED)

Total questions: 200
Q: Jak nazywa się pierwsza litera alfabetu greckiego?
A: Alfa.
Expected: alfa
Correct!

Q: Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?
A: cięciwa
Expected: cięciwa
Correct!

Q: W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?
A: Egipt.
Expected: w Egipcie
Correct!

Q: Czy w państwach starożytnych powoływani byli posłowie i poselstwa?
A: Tak.
Expected: tak
Correct!

Q: W jakim zespole występowała Hanka w filmie „Żona dla Australijczyka”?
A: "Mazowsze"
Expected: Mazowsze
Correct!

Q: W którym państwie leży Bombaj?
A: Indyjskim mieście Bombaj (od 1995 r. noszącym nazwę Mumbaj)
Expected: w Indiach
Incorrect!

Q: Który numer boczny nosi czołg Rudy z „Czterech pancernych”?
A: 102.
Expected: 102
Correct!

Q: Co budował w Egipcie inżynier Tarkowski, ojciec Stasia?
A: Niestety żaden z kontekstów nie informuje o tym, kto to inżynier Tarkowski i jaki miał wpływ na Budowę w Egipcie.
Expected: Kanał Sueski
Incorrect!

Q: Czy owoce niektóryc